## Analysis correlating evt and raw

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import awkward as ak
import os
import pint
import glob
import re
from functools import partial

# I use the SiPM QC Analysis kernel (from .venv/bin)

u = pint.get_application_registry()

from lgdo import lh5
from lgdo.lh5 import LH5Store, LH5Iterator, read, read_as, ls, show
from dspeed.vis.waveform_browser import WaveformBrowser
from legendmeta import LegendMetadata

from latools.utils import get_key_for_rawid, get_detector_system_for_channelname, get_filtered_keys_in_detectorsystem
from latools import core
from latools.browse import BrowseTask, BrowseAnydetTask
from latools.histogram import HistogramTask, Histogram2DTask, CategoricalHistogramTask, CategoricalHistogram2DTask
from latools.counter import CountTask

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 14

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250624T140504Z")

In [ ]:
def gimme_raw_filename_from_dsp(raw_dir: str, dsp_dir: str, dspfilename: str) -> str:
    return dspfilename.replace(dsp_dir, raw_dir).replace("tier_dsp", "tier_raw")

def gimme_tier_filename_dict(raw_dir: str, dsp_dir: str) -> list[dict[str, str]]:
    """
    Returns a list of dictionaries with keys 'raw' and 'dsp' for each file in dsp_dir.
    The 'raw' key contains the corresponding raw file path.
    """
    dsp_files = glob.glob(dsp_dir + "/l200-*-tier_dsp.lh5")
    dsp_files.sort()
    return [
        {
            "raw": gimme_raw_filename_from_dsp(raw_dir, dsp_dir, dsp_file),
            "dsp": dsp_file
        }
        for dsp_file in dsp_files
    ]

my_dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p14r006dsp")
up_dsp_dir = os.path.join(proj_dir, "data/tier/dsp/lac/p14/r006")
raw_dir = os.path.join(proj_dir, "data/tier/raw/lac/p14/r006")

compile_up_arrays = partial(core.compile_arrays, tier_filename_dict=gimme_tier_filename_dict(raw_dir, up_dsp_dir))
compile_my_arrays = partial(core.compile_arrays, tier_filename_dict=gimme_tier_filename_dict(raw_dir, my_dsp_dir))

In [ ]:
# These 49 rawids I see in the event tier of p14 r006 lac. Some channels existing in raw & dsp (e.g. S054) don't show up here...
rawids_used_in_evt = [1052803, 1052804, 1054401, 1054402, 1054403, 1054404, 1054405, 1056000, 1056001, 1056003, 1056004, 1056005, 1057601, 1057602, 1057603, 1057604, 1057605, 1059200, 1059201, 1059202, 1059204, 1059205, 1060801, 1060803, 1060804, 1060805, 1062400, 1062401, 1062402, 1062403, 1062404, 1062405, 1064000, 1064001, 1064002, 1064003, 1064004, 1065600, 1065601, 1065602, 1065603, 1065604, 1065605, 1067200, 1067201, 1067202, 1067203, 1067204, 1067205]
usable_sipm_keys = get_filtered_keys_in_detectorsystem(chmap, "spms", rawids_used_in_evt)

In [ ]:
# set up chain's input data
inputArraysDef = []

def generate_SiPM_hit_defs():
    ret = []
    for sipm in ["S015"]: # usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [(f"{sipm}_wf_fwhm", f"{sipm}/dsp/wf_fwhm"), (f"{sipm}_wf_mode", f"{sipm}/dsp/wf_mode"), (f"{sipm}_wf_min_small", f"{sipm}/dsp/wf_min_small"), (f"{sipm}_wf_min", f"{sipm}/dsp/wf_min")]
        #ret += [(f"{sipm}_curr_fwhm", f"{sipm}/dsp/curr_fwhm"), (f"{sipm}_curr_mode", f"{sipm}/dsp/curr_mode"), (f"{sipm}_curr_min_small", f"{sipm}/dsp/curr_min_small")]
    return ret
inputArraysDef += generate_SiPM_hit_defs()

In [ ]:
# set up processing functions
genArrayDef = []

def generate_SiPM_hit_gen_defs():
    ret = []
    for sipm in ["S015"]: # usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [
            ([f"{sipm}_wf_min_small", f"{sipm}_wf_mode"], f"{sipm}_min_rel", lambda x: x[0] - x[1]),
            ([f"{sipm}_min_rel", f"{sipm}_wf_fwhm"], f"{sipm}_any_noise_class", lambda x: (-x[0])/(6*x[1])),
            #([f"{sipm}_curr_min_small", f"{sipm}_curr_mode"], f"{sipm}_curr_min_rel", lambda x: x[0] - x[1]),
            #([f"{sipm}_min_rel", f"{sipm}_curr_fwhm"], f"{sipm}_curr_any_noise_class", lambda x: (-x[0])/(6*x[1])),
        ]
    return ret
genArrayDef += generate_SiPM_hit_gen_defs()

In [ ]:
my_arrays = compile_my_arrays(inputArraysDef, genArrayDef)
up_arrays = compile_up_arrays(inputArraysDef, genArrayDef)

In [ ]:
core.oneshot([my_arrays["S015_wf_fwhm"], up_arrays["S015_wf_fwhm"]], Histogram2DTask(4,5,500,4,5,500, logz=True))

In [ ]:
core.oneshot([my_arrays["S015_wf_mode"], up_arrays["S015_wf_mode"]], Histogram2DTask(3730,3750,100,3730,3750,100, logz=True))

In [ ]:
core.oneshot([my_arrays["S015_wf_min_small"], up_arrays["S015_wf_min_small"]], Histogram2DTask(3710,3750,100,3710,3750,100, logz=True))
ak.all(my_arrays["S015_wf_min_small"] == up_arrays["S015_wf_min_small"])

In [ ]:
core.oneshot([my_arrays["S015_wf_min"], up_arrays["S015_wf_min"]], Histogram2DTask(3730,3750,100,3730,3750,100, logz=True))
ak.all(my_arrays["S015_wf_min"] == up_arrays["S015_wf_min"])